In [1]:
import keras
import warnings
warnings.simplefilter("ignore")
import sys
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from skimage.io import imread
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.
/home/garima/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/garima/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/garima/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/garima/.local/lib/python3.6/site-packages/tensorflow/python

In [ ]:
#run only when frame needs to be extracted
count = 0
videoFile = "train.mp4"
# capturing the video from the given path
cap = cv2.VideoCapture(videoFile) 
#here frame rate is 5(you can change it to any other number
frameRate = cap.get(5) 
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="medium%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

In [ ]:
#read csv files
train = pd.read_csv("./train.csv")
test = pd.read_csv('./test.csv')
#create array
train_image = []
test_image = []
#Dataloading for Train Images
for i in tqdm(range(train.shape[0])):
    img = image.load_img('ntrain/'+train['image_file'][i]+'.jpg', target_size=(360,640,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
 
#Dataloading for Test Images
for i in tqdm(range(test.shape[0])):
    img = image.load_img('ntest/'+test['image_file'][i]+'.jpg', target_size=(360,640,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test_array = np.array(test_image)
 
print("Test array created", test_array.shape)
print(X.shape)
 
#Labels
y=train['emotion'].values
y = to_categorical(y)

In [ ]:
pool_size = 2
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size= (4,4 ), activation='relu', input_shape= (360,640,1)))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
Dropout(0.2)
model.add(Conv2D(filters = 128, kernel_size= (3,3 ), activation='relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
Dropout(0.2)
model.add(Conv2D(filters = 256, kernel_size= (3,3 ), activation='relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
Dropout(0.2)
model.add(Conv2D(filters = 512, kernel_size= (3,3 ), activation='relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
Dropout(0.2)
model.add(Conv2D(filters = 512, kernel_size= (3,3 ), activation='relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
Dropout(0.2)    
model.add(Flatten())
model.add(Dense( units =128,activation='relu' ))
model.add(Dense( units =64,activation='relu' ))
model.add(Dense(units = 5, activation='softmax'))

In [ ]:

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
print(model)
model.fit(X, y, epochs=50)
model.save('./model.h5')
model.save_weights('./weights.h5')
prediction = model.predict_classes(test_array)
print(prediction)